In [44]:
import json
from pathlib import Path

In [41]:
# 需要用户输入的内容
path = './通用AVG'
header = '显示名'
meta = {
    "name":"通用AVG风格",
    "cover":"@/cover.jpg",
    "describe":"提供三种颜色选择，适用于安科、跑团、同人小剧场等，附一张怪物立绘。",
    "author":"GULILI",
    "licence":"可以用于任何用途，无需注明作者"
}

In [50]:
# 载入rgpj
workpath = Path(path)
find_result = list(workpath.glob('*.rgpj'))
if find_result:
    rplgen_project = json.load(open(find_result[0].absolute()))

In [5]:
# 创建一个空白的模板
rplgen_intel_template = {}

In [7]:
# 写入meta信息
# 封面要求：540p，jpg
rplgen_intel_template['meta'] = meta

In [11]:
# 项目配置，移除原有的项目名称和封面
config = rplgen_project['config'].copy()
config.pop('Name')
config.pop('Cover')
rplgen_intel_template['config'] = config


In [15]:
# 排序媒体
media_order = {
    'Pos'       : ['Pos','FreePos','PosGrid','BezierCurve'],
    'Text'      : ['Text', 'StrokeText', 'RichText', 'HPLabel'],
    'Animation' : ['Animation'],
    'Bubble'    : ['Bubble','Balloon','DynamicBubble','ChatWindow'],
    'Background': ['Background'],
    'Audio'     : ['Audio','BGM'],
}
static_media = {}
for type in media_order:
    this_type:list = media_order[type]
    for name in rplgen_project['mediadef']:
        item = rplgen_project['mediadef'][name]
        if item['type'] in this_type:
            static_media[name] = item

In [17]:
# 添加媒体
rplgen_intel_template['media'] = {
    'static':static_media,
    'dynamic':{}
}

In [25]:
# 检视角色
# 获取自定义列的信息
table_col = ['Name','Subtype','Animation','Bubble','Voice','SpeechRate','PitchRate']
custom = []
static_list = []
dynamic_list = []
for keyword in rplgen_project['chartab']:
    this_char = rplgen_project['chartab'][keyword]
    # 检视自定义
    for col in this_char:
        if col not in table_col and col not in custom:
            custom.append(col)
    # 检视名字
    name:str = this_char['Name']
    if name.isdigit():
        dynamic_list.append(keyword)
    else:
        static_list.append(keyword)

In [28]:
dynamic_charactor = {}
static_charactor = {}
# 静态角色采用的列
static_use_col = ['Name','Subtype','Animation','Bubble'] + custom
# 动态角色采用的列，不含header
dynamic_use_col = ['Animation','Bubble'] + custom
dynamic_use_col.remove(header)
# 获取静态角色
for keyword in static_list:
    this_char = rplgen_project['chartab'][keyword]
    tplt_char = {}
    name = this_char['Name']
    for col in static_use_col:
        tplt_char[col] = this_char[col]
    static_charactor[name] = tplt_char
# 获取动态角色
for keyword in dynamic_list:
    this_char = rplgen_project['chartab'][keyword]
    tplt_char = {}
    name = this_char['Name']
    for col in dynamic_use_col:
        tplt_char[col] = this_char[col]
    dynamic_charactor[name] = tplt_char

In [32]:
# 添加角色
rplgen_intel_template['charactor'] = {
    "custom":custom,
    "header":header,
    'static':static_charactor,
    'dynamic':dynamic_charactor
}

In [35]:
# 剧本
rplgen_intel_template['preset'] = rplgen_project['logfile']['preset']
if 'example' in rplgen_project['logfile']:
    rplgen_intel_template['example'] = rplgen_project['logfile']['example']
else:
    rplgen_intel_template['example'] = None

In [54]:
with open(workpath.absolute()/'main.rgint','w',encoding='utf-8') as rgint:
    json.dump(rplgen_intel_template,rgint,ensure_ascii=False,indent=4)